# Session 3: 양자 알고리즘 입문

## 🎯 학습 목표
- Deutsch 알고리즘으로 시작하기
- Grover 검색 알고리즘 체험
- 양자 병렬성 이해
- 실제 문제 해결 맛보기

In [ ]:
# 필요한 라이브러리
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.providers.fake_provider import GenericBackendV2
# from qiskit_ibm_runtime.fake_provider import FakeJakartaV2
from qiskit import transpile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

%matplotlib inline

backend = GenericBackendV2(num_qubits=5, noise_info=True)
# backend = FakeJakartaV2()
sampler = Sampler(backend)

print("양자 알고리즘의 세계로 오신 것을 환영합니다! ")

## Part 1: 양자 병렬성

### 1.1 고전 vs 양자 처리

In [ ]:
# 고전 컴퓨터: 하나씩 처리
def classical_function(x):
    """간단한 함수: f(x) = x mod 2"""
    return x % 2

print("고전 컴퓨터: 4개 입력을 하나씩 처리")
print("="*40)
for x in range(4):
    result = classical_function(x)
    print(f"f({x}) = {result}")
print("\n총 4번의 연산 필요!")

# 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# 고전 처리
ax1.set_title("Classical Computer: Sequential Processing", fontweight='bold')
for i in range(4):
    rect = Rectangle((i*1.5, 0), 1, 1, facecolor='lightgray', edgecolor='black')
    ax1.add_patch(rect)
    ax1.text(i*1.5 + 0.5, 0.5, f'{i:02b}', ha='center', va='center')
    if i < 3:
        ax1.arrow(i*1.5 + 1.1, 0.5, 0.3, 0, head_width=0.1, head_length=0.1, fc='red')
ax1.set_xlim(-0.5, 6)
ax1.set_ylim(-0.5, 1.5)
ax1.axis('off')
ax1.text(3, -0.3, "4 operations", ha='center', fontsize=12, color='red')

# 양자 처리
ax2.set_title("Quantum Computer: Parallel Processing", fontweight='bold')
rect = Rectangle((1, 0), 3, 1, facecolor='lightblue', edgecolor='blue', linewidth=2)
ax2.add_patch(rect)
ax2.text(2.5, 0.5, '00 + 01 + 10 + 11', ha='center', va='center', fontsize=11)
ax2.set_xlim(0, 5)
ax2.set_ylim(-0.5, 1.5)
ax2.axis('off')
ax2.text(2.5, -0.3, "1 operation!", ha='center', fontsize=12, color='blue')

plt.tight_layout()
plt.show()

### 1.2 양자 병렬성 구현

In [ ]:
# 양자 병렬성: 모든 입력을 동시에!
def quantum_parallelism_demo():
    """2큐비트로 4개 상태 동시 처리"""
    qc = QuantumCircuit(2)
    
    # 모든 가능한 입력의 중첩 생성
    qc.h(0)  # 첫 번째 큐비트: 0과 1의 중첩
    qc.h(1)  # 두 번째 큐비트: 0과 1의 중첩
    
    # 결과: |00⟩ + |01⟩ + |10⟩ + |11⟩
    
    return qc

qc = quantum_parallelism_demo()
print("양자 병렬성 회로:")
qc.draw('mpl')

# 상태 벡터 확인
state = Statevector(qc)
print("\n생성된 중첩 상태:")
for i, amplitude in enumerate(state.data):
    if abs(amplitude) > 0.001:
        print(f"|{i:02b}⟩: {amplitude:.3f}")

print("\n💡 2개 게이트로 4개 상태를 동시에 준비했습니다!")

## Part 2: Deutsch 알고리즘 - 첫 양자 알고리즘!

### 2.1 문제 설정

In [ ]:
print("Deutsch 알고리즘: 함수의 성질 알아내기")
print("="*50)
print("블랙박스 함수 f: {0,1} → {0,1}")
print("\n가능한 함수들:")

functions = [
    ("상수 함수 1", "f(0)=0, f(1)=0", "항상 0"),
    ("상수 함수 2", "f(0)=1, f(1)=1", "항상 1"),
    ("균형 함수 1", "f(0)=0, f(1)=1", "입력 그대로"),
    ("균형 함수 2", "f(0)=1, f(1)=0", "입력 반전")
]

for name, values, desc in functions:
    print(f"  • {name}: {values} ({desc})")

print("\n문제: 함수가 상수인지 균형인지 판별하기")
print("고전: 2번 평가 필요 (f(0)과 f(1) 모두 확인)")
print("양자: 1번 평가로 충분!")

### 2.2 Deutsch 알고리즘 구현

In [ ]:
def deutsch_algorithm(oracle_type='balanced_id'):
    """
    Deutsch 알고리즘 구현
    oracle_type: 'constant_0', 'constant_1', 'balanced_id', 'balanced_not'
    """
    # 2큐비트: 입력(q0), 보조(q1)
    qc = QuantumCircuit(2, 1)
    
    # Step 1: 초기화
    qc.x(1)  # 보조 큐비트를 |1⟩로
    
    # Step 2: Hadamard 게이트
    qc.h(0)
    qc.h(1)
    
    qc.barrier()
    
    # Step 3: Oracle 적용
    if oracle_type == 'constant_0':
        # f(x) = 0: 아무것도 안 함
        pass
    elif oracle_type == 'constant_1':
        # f(x) = 1: 보조 큐비트 반전
        qc.x(1)
    elif oracle_type == 'balanced_id':
        # f(x) = x: CNOT
        qc.cx(0, 1)
    elif oracle_type == 'balanced_not':
        # f(x) = NOT x: X + CNOT
        qc.x(0)
        qc.cx(0, 1)
        qc.x(0)
    
    qc.barrier()
    
    # Step 4: Hadamard (입력 큐비트만)
    qc.h(0)
    
    # Step 5: 측정
    qc.measure(0, 0)
    
    return qc

# 모든 Oracle 테스트
oracle_types = [
    ('constant_0', 'Constnat (f=0)'),
    ('constant_1', 'Constant (f=1)'),
    ('balanced_id', 'Balanced (f=x)'),
    ('balanced_not', 'Balanced (f=NOT x)')
]

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()

for idx, (oracle, title) in enumerate(oracle_types):
    # 알고리즘 실행
    qc = deutsch_algorithm(oracle)
    
    # 시뮬레이션
    transpiled = transpile(qc, backend)
    job = sampler.run([(transpiled, [])], shots=1000)
    result = job.result()
    counts = result[0].data.c.get_counts()
    print(counts)
    
    # 시각화
    ax = axes[idx]
    bars = ax.bar(counts.keys(), counts.values())
    
    # 색상 설정
    if counts.get('0', 0) > counts.get('1', 0):
        bars[0].set_color('blue')
        result_text = "Result: 0 → Constant Function"
    else:
        bars[0].set_color('red')
        result_text = "Result: 1 → Balanced Function"
    
    ax.set_title(title, fontweight='bold')
    ax.set_xlabel('Measurement Result')
    ax.set_ylabel('Counts')
    ax.set_ylim(0, 1100)
    ax.text(0.5, 0.9, result_text, ha='center', 
           transform=ax.transAxes, fontsize=11,
           bbox=dict(boxstyle='round', facecolor='wheat'))

plt.suptitle('Deutsch Alogorithm: Distinguish Function with One Measurement!', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("• 측정값 0 → 상수 함수")
print("• 측정값 1 → 균형 함수")
print("• 단 1번의 Oracle 호출로 판별 완료!")

## Part 3: Grover 검색 알고리즘

### 3.1 문제: 정렬되지 않은 데이터베이스 검색

In [ ]:
print("Grover 알고리즘: 양자 검색")
print("="*50)
print("문제: 4개 항목 중 1개 찾기")
print("\n데이터베이스:")

# 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 데이터베이스
items = ['00', '01', '10', '11']
target = 2  # '10'을 찾자

# 고전 검색
ax1.set_title("Classical Search: Sequential Scan", fontweight='bold')
for i, item in enumerate(items):
    color = 'gold' if i == target else 'lightgray'
    edgecolor = 'green' if i == target else 'black'
    rect = Rectangle((i*1.2, 0), 1, 1, facecolor=color, 
                    edgecolor=edgecolor, linewidth=2)
    ax1.add_patch(rect)
    ax1.text(i*1.2 + 0.5, 0.5, item, ha='center', va='center', fontsize=12)
    
    if i <= target:
        ax1.text(i*1.2 + 0.5, -0.3, 'check', ha='center', fontsize=9, color='red')

ax1.set_xlim(-0.2, 5)
ax1.set_ylim(-0.5, 1.5)
ax1.axis('off')
ax1.text(2, 1.3, f"Avg ~ {len(items)//2}, Worst = {len(items)}", 
        ha='center', fontsize=11, color='red')

# 양자 검색
ax2.set_title("Grover Search: Amplitude Amplification", fontweight='bold')
amplitudes = [0.2, 0.2, 0.9, 0.2]  # Grover 후
bars = ax2.bar(items, amplitudes)
bars[target].set_color('gold')
bars[target].set_edgecolor('green')
bars[target].set_linewidth(3)

ax2.set_xlabel('Item')
ax2.set_ylabel('Probability Amplitude')
ax2.set_ylim(0, 1)
ax2.text(0.5, 0.9, f"~{int(np.sqrt(len(items)))} iteration(s)", 
        ha='center', transform=ax2.transAxes, 
        fontsize=11, color='blue',
        bbox=dict(boxstyle='round', facecolor='lightblue'))

plt.tight_layout()
plt.show()

print(f"\n고전: O(N) = {len(items)}번")
print(f"양자: O(√N) = {int(np.sqrt(len(items)))}번")
print("\n속도 향상이 제곱근 수준입니다!")

### 3.2 Grover 알고리즘 구성 요소

In [ ]:
def grover_oracle(marked_item):
    """
        Oracle: 찾는 항목에 위상 반전
    """
    oracle = QuantumCircuit(2)
    
    if marked_item == 0:  # |00⟩
        oracle.x([0, 1])
        oracle.cz(0, 1)
        oracle.x([0, 1])
    elif marked_item == 1:  # |01⟩
        oracle.x(1)
        oracle.cz(0, 1)
        oracle.x(1)
    elif marked_item == 2:  # |10⟩
        oracle.x(0)
        oracle.cz(0, 1)
        oracle.x(0)
    else:  # |11⟩
        oracle.cz(0, 1)
    
    return oracle

def grover_diffuser():
    """Diffuser: 평균에 대한 반전"""
    diffuser = QuantumCircuit(2)
    
    # H 게이트
    diffuser.h([0, 1])
    # X 게이트
    diffuser.x([0, 1])
    # CZ 게이트
    diffuser.cz(0, 1)
    # X 게이트
    diffuser.x([0, 1])
    # H 게이트
    diffuser.h([0, 1])
    
    return diffuser

print("Grover 알고리즘의 두 가지 핵심 요소:")
print("\n1. Oracle (찾는 항목 표시):")
oracle = grover_oracle(2)  # |10⟩ 찾기
oracle.draw('mpl')

print("\n2. Diffuser (진폭 증폭):")
diffuser = grover_diffuser()
diffuser.draw('mpl')

### 3.3 완전한 Grover 알고리즘

In [ ]:
def grover_search(marked_item=2):
    """
        2큐비트 Grover 검색 (4개 중 1개 찾기)
    """
    
    qc = QuantumCircuit(2, 2)
    
    # Step 1: 초기화 (균등 중첩)
    qc.h([0, 1])
    qc.barrier()
    
    # Step 2: Grover 연산자 (Oracle + Diffuser)
    # 2큐비트의 경우 1번 반복이 최적
    
    # Oracle
    oracle = grover_oracle(marked_item)
    qc.compose(oracle, inplace=True)
    qc.barrier()
    
    # Diffuser
    diffuser = grover_diffuser()
    qc.compose(diffuser, inplace=True)
    qc.barrier()
    
    # Step 3: 측정
    qc.measure([0, 1], [0, 1])
    
    return qc

# Grover 알고리즘 실행
print(f"목표: |10⟩ (인덱스 2) 찾기")
print("="*40)

grover_circuit = grover_search(marked_item=2)
print("\nGrover 검색 회로:")
grover_circuit.draw('mpl')
plt.show()

# 시뮬레이션
transpiled = transpile(grover_circuit, backend)
job = sampler.run([(transpiled, [])], shots=1000)
result = job.result()
counts = result[0].data.c.get_counts()

# 결과 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 측정 결과
outcomes = ['00', '01', '10', '11']
measured = [counts.get(i, 0) for i in outcomes]
bars1 = ax1.bar(outcomes, measured)
bars1[2].set_color('gold')
bars1[2].set_edgecolor('green')
bars1[2].set_linewidth(3)

ax1.set_xlabel('State')
ax1.set_ylabel('Counts')
ax1.set_title('Grover Search Result', fontweight='bold')
ax1.set_xticks(outcomes)
ax1.set_xticklabels(['00', '01', '10', '11'])

# 성공률
success_rate = counts.get('10', 0) / 1000
ax2.pie([success_rate, 1-success_rate], 
        labels=[f'Found\n({success_rate:.1%})', f'Not Found\n({1-success_rate:.1%})'],
        colors=['green', 'lightgray'],
        autopct='%1.0f%%',
        startangle=90)
ax2.set_title('Search Success Rate', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n 결과: |10⟩이 {success_rate:.1%} 확률로 측정됨!")
print("단 1번의 Grover 반복으로 높은 성공률 달성!")

### 3.4 더 큰 데이터베이스

In [ ]:
# 3큐비트 Grover (8개 중 1개 찾기)
def grover_3qubit(marked=5):
    """3큐비트 Grover 검색"""
    qc = QuantumCircuit(3, 3)
    
    # 초기화
    qc.h([0, 1, 2])
    
    # 최적 반복 횟수: floor(π/4 * √8) ≈ 2
    iterations = 2
    
    for _ in range(iterations):
        # Oracle (|101⟩ = 5 찾기)
        qc.x(1)  # NOT on qubit 1
        qc.ccz(0, 1, 2)  # Multi-controlled Z
        qc.x(1)  # Restore
        
        # Diffuser
        qc.h([0, 1, 2])
        qc.x([0, 1, 2])
        qc.ccz(0, 1, 2)
        qc.x([0, 1, 2])
        qc.h([0, 1, 2])
    
    qc.measure([0, 1, 2], [0, 1, 2])
    return qc

# 실행
print("3큐비트 Grover: 8개 중 1개 찾기")
print(f"목표: |101⟩ (숫자 5)")
print(f"반복 횟수: 2 (최적)")

qc_3qubit = grover_3qubit()
transpiled = transpile(qc_3qubit, backend, optimization_level=2)
job = sampler.run([(transpiled, [])], shots=1000)
result = job.result()
counts = result[0].data.c.get_counts()

# 상위 결과만 표시
top_results = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:4]

print("\n상위 4개 결과:")
for outcome, count in top_results:
    marker = "⭐" if outcome == 5 else ""
    print(f"  |{outcome}>: {count:3d}회 {marker}")

success_rate = counts.get("101", 0) / 1000
print(f"\n성공률: {success_rate:.1%}")
print(f"고전 평균: {8/2}번 vs Grover: 2번")

In [ ]:
print("🎊 Session 3 완료!")
print("\n축하합니다! 당신은 이제:")
print("✅ 양자 병렬성을 이해합니다")
print("✅ Deutsch 알고리즘을 구현할 수 있습니다")
print("✅ Grover 검색을 사용할 수 있습니다")
print("✅ 양자 우위를 체험했습니다")
print("\n다음 단계:")
print("• 더 복잡한 알고리즘 (Shor, QFT)")
print("• 실제 양자 하드웨어 사용")
print("\n양자 컴퓨팅의 미래를 만들어가세요!")